## ENV SETUP

1. Install uv (or do it you're own way)
2. Run `uv sync`
3. Run `source .venv/bin/activate`

You're good to go.

# Instructions

The Task : Create the best CadQuery code generator model. 

1. Load the dataset (147K pairs of Images/CadQuery code).
2. Create a baseline model and evaluate it with the given metrics.
3. Enhance by any manner the baseline model and evaluate it again.
4. Explain you choices and possible bottlenecks. 
5. Show what enhancements you would have done if you had more time.

You can do *WHATEVER* you want, be creative, result is not what matters the most. 
Creating new model architectures, reusing ones you used in the past, fine-tuning, etc...

If you are GPU poor, there are solutions. Absolute value is not what matters, relative value between baseline and enhanced model is what matters.

In [1]:
from datasets import load_dataset
ds = load_dataset("CADCODER/GenCAD-Code", num_proc=16, split=["train", "test"], cache_dir="./hf_cache")

/home/boustilafiras/mecagent-technical-test/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Afficher les features et leurs types pour chaque dataset (train et test) sans surcharger la mémoire
for i, split in enumerate(["train", "test"]):
    print(f"Features et types pour le dataset '{split}':")
    sample = ds[i][0]
    for col in ds[i].column_names:
        print(f"- {col}: {type(sample[col])} (exemple: {repr(sample[col])[:100]})")
    print()

# Afficher les features détaillées (schéma HuggingFace) pour chaque split
for i, split in enumerate(["train", "test"]):
    print(f"Schéma détaillé (features) pour le dataset '{split}':")
    print(ds[i].features)
    print()

Features et types pour le dataset 'train':
- image: <class 'PIL.PngImagePlugin.PngImageFile'> (exemple: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=448x448 at 0x7B718AF62E50>)
- deepcad_id: <class 'str'> (exemple: '0000/00006371')
- cadquery: <class 'str'> (exemple: 'import cadquery as cq\n# Generating a workplane for sketch 0\nwp_sketch0 = cq.Workplane(cq.Plane(cq)
- token_count: <class 'int'> (exemple: 1292)
- prompt: <class 'str'> (exemple: 'Generate the CADQuery code needed to create the CAD for the provided image. Just the code, no other)
- hundred_subset: <class 'bool'> (exemple: False)

Features et types pour le dataset 'test':
- image: <class 'PIL.PngImagePlugin.PngImageFile'> (exemple: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=448x448 at 0x7B718AF81950>)
- deepcad_id: <class 'str'> (exemple: '0000/00009254')
- cadquery: <class 'str'> (exemple: 'import cadquery as cq\n# Generating a workplane for sketch 0\nwp_sketch0 = cq.Workplane(cq.Plane(cq)
- token_count

## Evaluation Metrics

1. Valid Syntax Rate metric assess the validity of the code by executing and checking if error are returned.
2. Best IOU assess the similarity between the meshes generated by the code.

In [3]:
from metrics.valid_syntax_rate import evaluate_syntax_rate_simple
from metrics.best_iou import get_iou_best

In [4]:
## Example usage of the metrics
sample_code = """
height = 60.0
width = 80.0
thickness = 10.0
diameter = 22.0

# make the base
result = (
    cq.Workplane("XY")
    .box(height, width, thickness)
)
"""

sample_code_2 = """
 height = 60.0
 width = 80.0
 thickness = 10.0
 diameter = 22.0
 padding = 12.0

 # make the base
 result = (
     cq.Workplane("XY")
     .box(height, width, thickness)
     .faces(">Z")
     .workplane()
     .hole(diameter)
     .faces(">Z")
     .workplane()
     .rect(height - padding, width - padding, forConstruction=True)
     .vertices()
     .cboreHole(2.4, 4.4, 2.1)
 )
"""
codes = {
    "sample_code": sample_code,
    "sample_code_2": sample_code_2,
}
vsr = evaluate_syntax_rate_simple(codes)
print("Valid Syntax Rate:", vsr)
iou = get_iou_best(sample_code, sample_code_2)
print("IOU:", iou)

Valid Syntax Rate: 1.0
IOU: 0.5834943417057687
IOU: 0.5834943417057687


## Have Fun